# Seasonal MODEL SELECTION DS4A Project - Team 18 - Vaccine Acceptance

---
Authorship: Marie-anne

---

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pickle

import xgboost as xgb

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import CategoricalNB, GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

from sklearn.inspection import permutation_importance

In [ ]:
#Import dfs
features = pd.read_csv(os.path.join(os.getcwd(), 'Data/training_set_features.csv'))
labels = pd.read_csv(os.path.join(os.getcwd(), 'Data/training_set_labels.csv'))
imp_feat = pd.read_csv(os.path.join(os.getcwd(), 'Data/imputed_train_hot_encoded.csv'))
imp_feat_not_hot = pd.read_csv(os.path.join(os.getcwd(), 'Data/imputed_train.csv'))

In [ ]:
# set label index
labels.set_index('respondent_id', inplace=True)

In [ ]:
#IMPUTED 
imp_feat.set_index('Unnamed: 0', inplace=True)
imp_feat.sort_index(inplace=True)


In [ ]:
#merge_df options

merged_df = imp_feat.join(labels)
#merged_df = imp_feat_small.join(labels)


df_h1n1 = merged_df.reset_index(drop=True).drop(['h1n1_vaccine'], axis=1)
df_h1n1.shape

### Train test split

In [ ]:
X = df_h1n1.iloc[:, :-1]
y= df_h1n1.iloc[:,-1]

X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                    test_size=0.1,
                                                    stratify=y,
                                                    random_state=42
                                                 )
# get feature names
feature_names=list(X_train)

#check shape
print(X.shape)
X_train.shape

In [ ]:
# # IMPUTED Scaling and 
X_train = StandardScaler().fit_transform(X_train)
print(X_train.shape)
X_train

### GridSearch CV

In [ ]:
# Create model_dict

model_GSCV = dict()

model_GSCV ['LR'] = LogisticRegression()
model_GSCV['SVM'] = SVC()
model_GSCV['RF'] = RandomForestClassifier()
#model_GSCV['XGB'] = xgb.XGBClassifier(objective= 'binary:logistic') #see tuning nb


In [ ]:
# Define best_model:
def best_model(name, model):
    '''run standard scaler and gridsearch CV pipeline on models
    Args:
        -model: initiated model 
        -name : name of model as str
    return list of best estimator and table of results
    '''
    pipe = Pipeline([('scaler', StandardScaler()), ('classifier',model)])
    best_model_stack = list()
    results_cv = dict()
    def grid_csv(params):
        
        GSCV = GridSearchCV(pipe, param_grid = params, scoring = ['accuracy', 'roc_auc', 'average_precision'], refit='average_precision', cv = 5, n_jobs=-1, verbose=True)
        best_clf = GSCV.fit(X_train, y_train)
        best_hyperparams = best_clf.best_params_
        best_score = best_clf.best_score_
        estimator = best_clf.best_estimator_
        print(best_score, best_hyperparams, estimator)
        table = best_clf.cv_results_
        results_cv[name] = table
        return name, best_hyperparams
    
    if name == 'LR':
        params = {'classifier__penalty' : ['l1', 'l2', 'elasticnet', 'none'], 
                  'classifier__C' : [0.2, 0.5, 1]} 
        best_model_stack.append(grid_csv(params))
        
    
    if name == 'SVM':
        params = {'classifier__kernel' : ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
                 'classifier__C' : [0.2, 0.5, 1]} 
        best_model_stack.append(grid_csv(params))


    if name == 'RF': 
        params = {'classifier__max_depth' : np.arange(100, 200, 50),
                  'classifier__criterion' : ['gini', 'entropy'],
                  'classifier__max_depth' : np.arange(5, 15, 1),
                 } 
        best_model_stack.append(grid_csv(params))
    
    if name == 'XGB':
            pass #for XGB fine tuning in XGB_tuning
        
    return best_model_stack, results_cv

In [ ]:
results_best_model = list()
scoring = dict()
for name, model in model_GSCV.items():
    scores = best_model(name, model)
    results_best_model.append(scores[0])
    scoring[name] = pd.DataFrame(scores[1][name])

In [ ]:
#save params
with open('Results/gridsearch_seasonal.txt', 'w') as file:
    file.write(str(results_best_model))

In [ ]:
with pd.ExcelWriter('Results/results_seasonal_CV.xls') as writer:
    for df_name, df in scoring.items():
        df.to_excel(writer, sheet_name=df_name)

### KFold CV

In [ ]:
# Create model_dict with gridsearchCV params

models = dict()

models['LR'] = LogisticRegression(C=0.2, penalty='none')
models['SVM'] = SVC(kernel='rbf', C=0.5)
models['RF'] = RandomForestClassifier(criterion= 'entropy', max_depth=11, n_jobs=-1)


In [ ]:
def evaluate_model(scoremodel):
    '''Calculate choosen score for different models using repeated stratified Kfold
    Args:
    - model : model_name(params)
    - score : metrics as string
    
    return scores
    '''
    cv = RepeatedStratifiedKFold(n_splits = 10, n_repeats=3, random_state=42)

  # Calculate accuracy using `cross_val_score()
    scores = cross_val_score(model, X_train, y_train, scoring=score, cv=cv, n_jobs=-1, error_score='raise', verbose=2)
    print(X_train.shape)
    return scores

In [ ]:
#quick test
np.mean(evaluate_model(LogisticRegression(C=1)))

In [ ]:
# Evaluate the models and store results
results = list()

for name, model in models.items():   
    scores = evaluate_model(model)
    results.append(scores)
    print('>%s %.3f (%.3f)' %(name, np.mean(scores), np.std(scores)))